In [1]:
using CSV, JuMP, Ipopt, DataFrames

In [2]:
D_params = [
    -0.0039842 199.1828;
    -0.0037 214.15
];
df2 = CSV.read("initial_production.csv", nullable=false)
df = CSV.read("supply.csv", nullable=false)
nFirms, _ = size(df)
nPeriods = 10
df

,country,reserves,capacity,marginal_cost
1,Saudi Arabia,108000,12000,9
2,Iran,41400,4600,10
3,Iraq,33300,3700,16
4,Kuwait,29700,3300,13
5,UAE,27000,3000,5
6,Venezuela,39600,4400,20
7,Nigeria,24300,2700,7


In [3]:
m = Model(solver = IpoptSolver())

@variable(m, q[1:nFirms, 1:nPeriods] >= 0)

@constraint(m, [sum(q[i, :]) for i = 1:nFirms] .<= df[:, 2])
@constraint(m, q[:, i=1:nPeriods] .<= df[:, 3])

totalQ = [sum(q[:, i]) for i = 1:nPeriods]

P = [dot(D_params[((i-1)%2) + 1, :], [totalQ[i], 1]) for i = 1:nPeriods]

Revenue = q .* P'
Costs = q .* df[:, 4]
RawProfits = Revenue - Costs
InterestMultiplier = [(1.05)^(11-i) for i = 1:10]
InterestAdjustedProfits = RawProfits .* InterestMultiplier'
PeriodProfits = [sum(InterestAdjustedProfits[i, :]) for i = 1:nFirms]
LeftoverQuantities = df[:, 2] - [sum(q[i, :]) for i = 1:nFirms]
BackstopProfits = LeftoverQuantities.*(70 - df[:, 4])
Profits = PeriodProfits + BackstopProfits

TotalOPECProfit = sum(Profits)

@variable(m, 0 <= badround <= 1)
@variable(m, 0 <= base[1:2] <= 1)
@variable(m, decr)

@constraint(m, q[:, 3] .== badround * df[:, 3])
@constraint(m, P[3] == 104)

for j = 4:nPeriods
    @constraint(m, q[:, j] .== (base[(j%2) + 1] - decr*(j-4))*df[:, 3])
end

for i = 1:nFirms
    for j = 1:2
        @constraint(m, q[i, j] == df2[i, 1+j])
    end
end

@objective(m, Max, sum(Profits))

solve(m)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      182
Number of nonzeros in inequality constraint Jacobian.:      140
Number of nonzeros in Lagrangian Hessian.............:      490

Total number of variables............................:       74
                     variables with only lower bounds:       70
                variables with lower and upper bounds:        3
                     variables with only upper bounds:        0
Total number of equa

:Optimal

In [4]:
getobjectivevalue(m)

3.6056239782891706e7

In [5]:
qdf = DataFrame(getvalue(q))
qdf[:country] = df[:, 1]
qdf

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,country
1,8300.0,12000.0,8506.848341751102,7757.849091398888,6453.042239778744,7535.73572183841,6230.928870218267,7313.622352277933,6008.81550065779,7091.508982717456,Saudi Arabia
2,4600.0,4600.0,3260.958531004589,2973.8421517029074,2473.666191915185,2888.698693371391,2388.522733583669,2803.555235039875,2303.3792752521526,2718.411776708358,Iran
3,2775.0,3700.0,2622.9449053732565,2392.003469847991,1989.6880239317798,2323.5185142335104,1921.203068317299,2255.0335586190295,1852.7181127028184,2186.548603004549,Iraq
4,3300.0,3300.0,2339.383293981553,2133.4085001346943,1774.5866159391549,2072.3273235055626,1713.5054393100233,2011.2461468764313,1652.424262680892,1950.1649702473,Kuwait
5,3000.0,2000.0,2126.7120854377754,1939.462272849722,1613.260559944686,1883.9339304596026,1557.7322175545667,1828.4055880694832,1502.2038751644475,1772.877245679364,UAE
6,4400.0,4400.0,3119.1777253087375,2844.544666846259,2366.115487918873,2763.1030980074174,2284.6739190800313,2681.6615291685757,2203.2323502411896,2600.2199603297336,Venezuela
7,2700.0,2700.0,1914.0408768939978,1745.51604556475,1451.9345039502175,1695.5405374136424,1401.9589957991102,1645.565029262535,1351.9834876480027,1595.5895211114275,Nigeria


In [6]:
getvalue(base), getvalue(decr)

([0.646487, 0.547008], 0.009254723731686567)